In [1]:
!pip install umap-learn
!pip install plotly

In [2]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.7.0 which is incompatible.


In [3]:
from google.colab import userdata
OPENAI_KEY = userdata.get('OPENAI_KEY')

In [4]:
# Sample archetype data
archetypes = {
    "Hero": "A brave and noble character who embarks on a journey of self-discovery and faces challenges to achieve a greater good.",
    "Mentor": "A wise and experienced guide who provides knowledge, advice, and support to help others grow and develop.",
    "Trickster": "A clever and mischievous character who challenges convention through wit, deceit, and manipulation.",
    "Shadow": "A dark reflection of the protagonist, representing repressed qualities or destructive tendencies.",
    "Sage": "An enlightened figure who has transcended ordinary understanding and shares profound wisdom.",
    "Guardian": "A protective figure who tests the hero and defends important boundaries or treasures.",
    "Creator": "An innovative and artistic character driven to bring new things into being.",
    "Explorer": "An adventurous soul who seeks out new experiences and pushes beyond known boundaries.",
    "Innocent": "A pure-hearted character whose optimism and moral clarity shine even in dark times.",
    "Rebel": "A character who challenges existing systems and fights against established norms."
}

In [5]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

In [12]:
import numpy as np
import pandas as pd
from openai import OpenAI

def get_embeddings(texts, model="text-embedding-3-small"):
    """Get embeddings from OpenAI API"""
    embeddings = []

    for text in texts:
        response = client.embeddings.create(
            model=model,
            input=text
        )
        embeddings.append(response.data[0].embedding)

    return np.array(embeddings)

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

dummyEmbeddings = {}
for archetype, description in archetypes.items():
    embedding = get_embedding(f'{archetype}: {description}')
    dummyEmbeddings[archetype] = embedding

print(dummyEmbeddings)

{'Hero': [-0.015925278887152672, -0.023748017847537994, -0.010830588638782501, 0.052229318767786026, 0.0024045305326581, -0.04404517263174057, -0.019096344709396362, 0.030265023931860924, 0.05227595195174217, 0.04707633703947067, -0.003614082233980298, 0.019282877445220947, -0.03763309121131897, -0.04357883706688881, 0.053954750299453735, 0.026207925751805305, -0.06304825097322464, 0.0005180670414119959, -0.017359253019094467, 0.041969988495111465, 0.013628588058054447, 0.04313582181930542, -0.0016277943504974246, 0.042716123163700104, 0.01172828022390604, 0.018909811973571777, -0.05899114906787872, 0.045654021203517914, 0.06496021151542664, -0.036000922322273254, 0.06491357833147049, -0.011436821892857552, 0.02737375907599926, 0.000489285564981401, -0.003383830189704895, -0.012719238176941872, -0.003060311544686556, -0.07232827693223953, 0.027886725962162018, 0.0047711716033518314, -0.009454905986785889, -0.07969634234905243, 0.02578822523355484, 0.033436089754104614, -0.0070707770064

In [34]:
import umap.umap_ as umap

def doUMAP(descriptions, embeddings):
    reducer = umap.UMAP(random_state=42)
    embedding_2d = reducer.fit_transform(embeddings)

    df = pd.DataFrame({
        'Archetype': descriptions.keys(),
        'Description': descriptions.values(),
        'x': embedding_2d[:, 0],
        'y': embedding_2d[:, 1]
    })
    return df

In [14]:
import plotly.express as px
import umap.umap_ as umap

def create_visualization(descriptions, embeddings):
    # Reduce dimensionality with UMAP
    reducer = umap.UMAP(random_state=42)
    embedding_2d = reducer.fit_transform(embeddings)

    # Create DataFrame for plotting
    df = pd.DataFrame({
        'Archetype': descriptions.keys(),
        'Description': descriptions.values(),
        'x': embedding_2d[:, 0],
        'y': embedding_2d[:, 1]
    })

    # Create interactive plot
    fig = px.scatter(
        df,
        x='x',
        y='y',
        text='Archetype',
        title='Literary Archetypes in Semantic Space',
        hover_data=['Description']
    )

    # Update layout
    fig.update_traces(
        textposition='top center',
        marker=dict(size=10)
    )
    fig.update_layout(
        showlegend=False,
        hovermode='closest'
    )

    return fig

embeddingValues = list(dummyEmbeddings.values())
fig = create_visualization(archetypes, embeddingValues)
fig.show()

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


In [15]:
fantasyArchetypes = [
  {
    "title": "The Chosen One",
    "description": "A seemingly ordinary person destined for extraordinary things, often marked by prophecy or unique abilities",
    "characters": ["Harry Potter", "Frodo Baggins", "Paul Atreides"]
  },
  {
    "title": "The Wise Mentor",
    "description": "An elderly sage who guides the hero with wisdom and sometimes magic",
    "characters": ["Gandalf", "Dumbledore", "Obi-Wan Kenobi"]
  },
  {
    "title": "The Dark Lord",
    "description": "A powerful evil being seeking domination or destruction of the world",
    "characters": ["Sauron", "Voldemort", "The Night King"]
  },
  {
    "title": "The Loyal Friend",
    "description": "A steadfast companion who supports the hero through their journey",
    "characters": ["Samwise Gamgee", "Ron Weasley", "Chewbacca"]
  },
  {
    "title": "The Warrior Princess",
    "description": "A noble woman who defies traditional roles to become a fierce fighter",
    "characters": ["Eowyn", "Xena", "Wonder Woman"]
  },
  {
    "title": "The Trickster",
    "description": "A clever and mischievous character who creates chaos but sometimes helps the hero",
    "characters": ["Loki", "Puck", "Jack Sparrow"]
  },
  {
    "title": "The Fallen Hero",
    "description": "A once-noble character corrupted by power or tragedy",
    "characters": ["Anakin Skywalker", "Boromir", "Arthas Menethil"]
  },
  {
    "title": "The Royal Heir",
    "description": "A legitimate or hidden heir to the throne who must reclaim their birthright",
    "characters": ["Aragorn", "Daenerys Targaryen", "Simba"]
  },
  {
    "title": "The Mysterious Stranger",
    "description": "A enigmatic figure with unclear motives who appears at crucial moments",
    "characters": ["Strider", "Tom Bombadil", "The Man in Black"]
  },
  {
    "title": "The Evil Advisor",
    "description": "A manipulative counselor who poisons the mind of those in power",
    "characters": ["Grima Wormtongue", "Jafar", "Littlefinger"]
  },
  {
    "title": "The Knight Errant",
    "description": "A wandering warrior bound by a personal code of honor",
    "characters": ["The Witcher", "Don Quixote", "The Mandalorian"]
  },
  {
    "title": "The Child of Prophecy",
    "description": "A young person whose birth or existence is tied to an important prediction",
    "characters": ["Anakin Skywalker", "Rand al'Thor", "Azor Ahai"]
  },
  {
    "title": "The Reluctant Hero",
    "description": "Someone who initially refuses the call to adventure but eventually accepts their destiny",
    "characters": ["Bilbo Baggins", "Neo", "Luke Skywalker"]
  },
  {
    "title": "The Wise Woman",
    "description": "A powerful female figure who provides guidance through magic or wisdom",
    "characters": ["Galadriel", "Minerva McGonagall", "Moiraine Damodred"]
  },
  {
    "title": "The Beast Master",
    "description": "A character with a special connection to animals or magical creatures",
    "characters": ["Hagrid", "Daine Sarrasri", "Eliza Thornberry"]
  },
  {
    "title": "The Dark Witch",
    "description": "A malevolent female magic user who often serves as an antagonist",
    "characters": ["Bellatrix Lestrange", "The White Witch", "Maleficent"]
  },
  {
    "title": "The Scholar Mage",
    "description": "A studious magic user who relies on knowledge and research",
    "characters": ["Hermione Granger", "Raistlin Majere", "Doctor Strange"]
  },
  {
    "title": "The Cursed One",
    "description": "A character suffering from a magical affliction or transformation",
    "characters": ["Beast", "Edward Scissorhands", "The Hulk"]
  },
  {
    "title": "The Wild Child",
    "description": "A character raised away from civilization, often by animals or magical beings",
    "characters": ["Mowgli", "Tarzan", "Peter Pan"]
  },
  {
    "title": "The Dragon Rider",
    "description": "A warrior who shares a special bond with a dragon mount",
    "characters": ["Eragon", "Daenerys Targaryen", "Hiccup"]
  },
  {
    "title": "The Shadow Assassin",
    "description": "A deadly killer who operates from the shadows, often with magical abilities",
    "characters": ["Artemis Entreri", "Grey Worm", "Azrael"]
  },
  {
    "title": "The Gentle Giant",
    "description": "A physically imposing character with a kind and peaceful nature",
    "characters": ["Hodor", "Hagrid", "The BFG"]
  },
  {
    "title": "The Lost Prince",
    "description": "A royal child separated from their heritage who must discover their true identity",
    "characters": ["Arthur Pendragon", "Jon Snow", "Perseus"]
  },
  {
    "title": "The Forest Guardian",
    "description": "A mystical protector of nature and wildlife",
    "characters": ["Radagast", "Pocahontas", "The Lorax"]
  },
  {
    "title": "The Battle Mage",
    "description": "A warrior who combines martial prowess with magical abilities",
    "characters": ["Gandalf", "Elminster", "Doctor Strange"]
  },
  {
    "title": "The Shapeshifter",
    "description": "A being capable of changing their physical form at will",
    "characters": ["Mystique", "Beorn", "Professor McGonagall"]
  },
  {
    "title": "The Prophecy Keeper",
    "description": "A character who maintains and interprets important predictions about the future",
    "characters": ["Trelawney", "The Oracle", "Melisandre"]
  },
  {
    "title": "The Dark Knight",
    "description": "A warrior who uses questionable methods to achieve noble goals",
    "characters": ["Geralt of Rivia", "Batman", "The Punisher"]
  },
  {
    "title": "The Fairy Godmother",
    "description": "A benevolent magical being who helps those in need",
    "characters": ["Cinderella's Godmother", "The Blue Fairy", "Flora, Fauna, and Merryweather"]
  },
  {
    "title": "The Cursed King",
    "description": "A ruler suffering from a magical affliction that affects their kingdom",
    "characters": ["Beast", "Fisher King", "Theoden"]
  },
  {
    "title": "The Wild Mage",
    "description": "A magic user whose powers are unpredictable and tied to nature",
    "characters": ["Daine Sarrasri", "Merlin", "Storm"]
  },
  {
    "title": "The Immortal Being",
    "description": "A character who cannot die and has lived through countless ages",
    "characters": ["Wolverine", "Elrond", "Vandal Savage"]
  },
  {
    "title": "The Elemental Master",
    "description": "A magic user who specializes in controlling one or more natural elements",
    "characters": ["Avatar Aang", "Storm", "Toph Beifong"]
  },
  {
    "title": "The Sacred Knight",
    "description": "A warrior devoted to a religious or spiritual cause",
    "characters": ["Galahad", "Templars", "Paladins"]
  },
  {
    "title": "The Rebel Leader",
    "description": "A charismatic figure who leads the resistance against tyranny",
    "characters": ["Princess Leia", "Katniss Everdeen", "Robin Hood"]
  },
  {
    "title": "The Blood Mage",
    "description": "A practitioner of forbidden magic that requires life force or sacrifice",
    "characters": ["Morgana", "Flemeth", "Blood Raven"]
  },
  {
    "title": "The Beast Slayer",
    "description": "A specialized warrior who hunts dangerous monsters",
    "characters": ["Van Helsing", "Beowulf", "Monster Hunter"]
  },
  {
    "title": "The Corrupted Priest",
    "description": "A religious figure who has fallen to darkness or evil",
    "characters": ["High Sparrow", "Claude Frollo", "Melisandre"]
  },
  {
    "title": "The Dream Walker",
    "description": "Someone who can enter and manipulate dreams",
    "characters": ["Morpheus", "Freddy Krueger", "Dream"]
  },
  {
    "title": "The Spirit Medium",
    "description": "A person who can communicate with the dead or spirit world",
    "characters": ["John Constantine", "Cole Sear", "Luna Lovegood"]
  },
  {
    "title": "The Enchantress",
    "description": "A female magic user who specializes in charm and illusion magic",
    "characters": ["Circe", "Morgan Le Fay", "The White Witch"]
  },
  {
    "title": "The Time Manipulator",
    "description": "A character who can control or travel through time",
    "characters": ["Doctor Strange", "Hermione (with Time-Turner)", "The Doctor"]
  },
  {
    "title": "The Soul Collector",
    "description": "A being who harvests or trades in souls",
    "characters": ["Mephisto", "Hades", "Death"]
  },
  {
    "title": "The Hidden Guardian",
    "description": "A protector who works in secret to maintain balance or safety",
    "characters": ["Nick Fury", "The Watchers", "Men in Black"]
  },
  {
    "title": "The Puppet Master",
    "description": "A manipulative character who controls others from behind the scenes",
    "characters": ["Littlefinger", "Emperor Palpatine", "Varys"]
  },
  {
    "title": "The War Chief",
    "description": "A leader who excels in military strategy and combat",
    "characters": ["Aragorn", "Theoden", "Leonidas"]
  },
  {
    "title": "The Witch Hunter",
    "description": "A specialist trained to track and eliminate magical threats",
    "characters": ["Solomon Kane", "Hansel", "Van Helsing"]
  },
  {
    "title": "The Rogue Wizard",
    "description": "A magic user who operates outside traditional magical society",
    "characters": ["Harry Dresden", "John Constantine", "Doctor Strange"]
  },
  {
    "title": "The Cursed Warrior",
    "description": "A fighter bearing a supernatural burden or curse",
    "characters": ["Ghost Rider", "Prince Ashitaka", "The Punisher"]
  },
  {
    "title": "The Divine Champion",
    "description": "A mortal chosen by the gods to carry out their will",
    "characters": ["Perseus", "Wonder Woman", "Thor"]
  },
  {
    "title": "The Shadow Queen",
    "description": "A female ruler who uses dark magic to maintain power",
    "characters": ["Evil Queen", "Ravenna", "Queen Bavmorda"]
  },
  {
    "title": "The Nature Spirit",
    "description": "A magical being embodying natural forces",
    "characters": ["Totoro", "Pan", "Dryads"]
  },
  {
    "title": "The Dimensional Traveler",
    "description": "A character who can move between different worlds or realities",
    "characters": ["Doctor Strange", "The Doctor", "Philip Pullman's Will"]
  },
  {
    "title": "The Cursed Lover",
    "description": "A character whose love is complicated by a magical affliction",
    "characters": ["Beast", "Edward Cullen", "Ladyhawke"]
  },
  {
    "title": "The Blood Hunter",
    "description": "A tracker who specializes in hunting vampires or other blood-drinking creatures",
    "characters": ["Blade", "Van Helsing", "Buffy Summers"]
  },
  {
    "title": "The Celestial Being",
    "description": "A powerful entity from the heavens or cosmic realm",
    "characters": ["Thor", "Gandalf", "The Silver Surfer"]
  },
  {
    "title": "The Chaos Agent",
    "description": "A character who deliberately creates disorder for their own purposes",
    "characters": ["Loki", "Q", "The Joker"]
  },
  {
    "title": "The Dream Weaver",
    "description": "A magic user who can create and manipulate illusions",
    "characters": ["Morpheus", "Sandman", "Inception's Architects"]
  },
  {
    "title": "The Soul Knight",
    "description": "A warrior who protects the spiritual realm",
    "characters": ["Ghost Rider", "Spawn", "Constantine"]
  },
  {
    "title": "The Storm Bringer",
    "description": "A being capable of controlling weather and atmospheric conditions",
    "characters": ["Storm", "Thor", "Zeus"]
  },
  {
    "title": "The Blood Queen",
    "description": "A female ruler who maintains power through blood magic or sacrifice",
    "characters": ["Elizabeth Bathory", "Queen of Hearts", "Crimson Queen"]
  },
  {
    "title": "The Void Walker",
    "description": "A character who can traverse or manipulate the empty spaces between realms",
    "characters": ["Doctor Strange", "Nightcrawler", "Portal's Chell"]
  },
  {
    "title": "The Summoner",
    "description": "A magic user who specializes in calling forth creatures or spirits",
    "characters": ["Yuna", "Rita Repulsa", "Pokemon Trainers"]
  },
  {
    "title": "The Living Weapon",
    "description": "A person transformed into a weapon through magic or other means",
    "characters": ["Winter Soldier", "Wolverine", "Edward Scissorhands"]
  },
  {
    "title": "The Star Child",
    "description": "A being of cosmic origin in human form",
    "characters": ["Superman", "Starman", "Star-Lord"]
  },
  {
    "title": "The Witch King",
    "description": "A male ruler who combines magical power with political authority",
    "characters": ["Witch-king of Angmar", "The Lich King", "The Night King"]
  },
  {
    "title": "The Memory Keeper",
    "description": "A character who preserves or manipulates memories",
    "characters": ["Dumbledore", "The Giver", "Memory Monks"]
  },
  {
    "title": "The Plague Doctor",
    "description": "A healer who deals with magical diseases and curses",
    "characters": ["Madam Pomfrey", "Doctor Strange", "Elrond"]
  },

  {
    "title": "The Eternal Champion",
    "description": "A warrior who is continuously reborn to fight evil across different ages",
    "characters": ["Link", "Michael Moorcock's Eternal Champion", "The Avatar"]
  },
  {
    "title": "The Tomb Raider",
    "description": "An adventurer who explores ancient magical ruins and artifacts",
    "characters": ["Indiana Jones", "Lara Croft", "Nathan Drake"]
  },
  {
    "title": "The Gatekeeper",
    "description": "A guardian who protects portals between worlds or realms",
    "characters": ["Heimdall", "The Guardian of Forever", "Hodor"]
  },
  {
    "title": "The Cosmic Horror",
    "description": "An ancient being whose very existence threatens reality",
    "characters": ["Cthulhu", "The Other Gods", "The Nothing"]
  },
  {
    "title": "The Undying King",
    "description": "A ruler who has achieved immortality through dark means",
    "characters": ["The Lich King", "Nagash", "The Night King"]
  },
  {
    "title": "The World Walker",
    "description": "A traveler who moves between different planes of existence",
    "characters": ["Rincewind", "Lyra Belacqua", "Richard Cypher"]
  },
  {
    "title": "The Chaos Mage",
    "description": "A spellcaster who harnesses unpredictable, wild magic",
    "characters": ["Rincewind", "Willow", "Wanda Maximoff"]
  },
  {
    "title": "The Dream Lord",
    "description": "A powerful being who rules over the realm of dreams",
    "characters": ["Morpheus", "The Sandman", "Hypnos"]
  },
  {
    "title": "The Soul Weaver",
    "description": "A magic user who can manipulate the essence of life itself",
    "characters": ["Doctor Facilier", "Shang Tsung", "Death"]
  },
  {
    "title": "The Forgotten God",
    "description": "A deity who has lost most of their worshippers and power",
    "characters": ["Wednesday", "Pan", "The Old Gods"]
  },
  {
    "title": "The Rune Master",
    "description": "A scholar who derives power from ancient magical symbols",
    "characters": ["Elminster", "Doctor Strange", "Gandalf"]
  },
  {
    "title": "The Living Legend",
    "description": "A hero whose deeds have become mythological in their own lifetime",
    "characters": ["Drizzt Do'Urden", "Conan", "King Arthur"]
  },
  {
    "title": "The Death Knight",
    "description": "A fallen warrior resurrected to serve dark powers",
    "characters": ["Arthas", "Lord Soth", "The Nazgûl"]
  },
  {
    "title": "The Spell Thief",
    "description": "A character who can steal or copy others' magical abilities",
    "characters": ["Rogue", "Sylar", "Peter Petrelli"]
  },
  {
    "title": "The Living Prophecy",
    "description": "A person whose very existence fulfills an ancient prediction",
    "characters": ["Neo", "John Connor", "Anakin Skywalker"]
  },
  {
    "title": "The Cosmic Trickster",
    "description": "A powerful being who uses chaos and mischief to test heroes",
    "characters": ["Q", "Mr. Mxyzptlk", "Coyote"]
  },
  {
    "title": "The Spirit Walker",
    "description": "A shaman who can journey through the spirit world",
    "characters": ["Aang", "John Constantine", "Doctor Strange"]
  },
  {
    "title": "The Cursed Prophet",
    "description": "A seer whose predictions are true but never believed",
    "characters": ["Cassandra", "Trelawney", "The Oracle"]
  },
  {
    "title": "The Elemental Lord",
    "description": "A being who embodies and controls a force of nature",
    "characters": ["Poseidon", "Gaia", "Mother Nature"]
  },
  {
    "title": "The Void Lord",
    "description": "A entity seeking to consume or nullify existence itself",
    "characters": ["Dormammu", "The Nothing", "Anti-Monitor"]
  }
]
print(len(fantasyArchetypes))

88


In [16]:
fantasyEmbeddings = {}
for i in fantasyArchetypes:
    text = f"{i['title']}: {i['description']}. Characters: {','.join(i['characters'])}"
    embedding = get_embedding(text)
    fantasyEmbeddings[i['title']] = embedding

print(fantasyEmbeddings)

{'The Chosen One': [0.01835699751973152, -0.020838357508182526, -0.024965517222881317, 0.02259809710085392, 0.017483457922935486, -0.05762831121683121, 0.005655854009091854, 0.04570259153842926, -0.00979250855743885, -0.01733153685927391, -0.0363595150411129, 0.006121109239757061, -0.044968314468860626, 0.0045449393801391125, 0.0379040353000164, 0.028662236407399178, -0.0777323916554451, 0.01682513765990734, -0.019306497648358345, 0.03230831399559975, -0.012881548143923283, 0.03456179425120354, 0.05028551444411278, 0.037195075303316116, -0.04747499153017998, 0.004412009380757809, -0.03147275373339653, 0.03159935399889946, 0.04195523262023926, 0.006766769103705883, 0.042714834213256836, -0.02901671640574932, 0.04977911338210106, -0.008064419031143188, -0.01102052815258503, -0.01774931699037552, -0.0031428446527570486, -0.013710778206586838, -0.006589529104530811, -0.018293697386980057, -0.03337175399065018, -0.04408211261034012, 0.028181156143546104, 0.013001818209886551, 0.034333914518

In [27]:
fantasyArchetypesKeysValues = {i['title']: i['description'] for i in fantasyArchetypes}
fantasyEmbeddingValues = list(fantasyEmbeddings.values())
print(fantasyArchetypesKeysValues)
fig = create_visualization(fantasyArchetypesKeysValues, fantasyEmbeddingValues)
fig.show()

{'The Chosen One': 'A seemingly ordinary person destined for extraordinary things, often marked by prophecy or unique abilities', 'The Wise Mentor': 'An elderly sage who guides the hero with wisdom and sometimes magic', 'The Dark Lord': 'A powerful evil being seeking domination or destruction of the world', 'The Loyal Friend': 'A steadfast companion who supports the hero through their journey', 'The Warrior Princess': 'A noble woman who defies traditional roles to become a fierce fighter', 'The Trickster': 'A clever and mischievous character who creates chaos but sometimes helps the hero', 'The Fallen Hero': 'A once-noble character corrupted by power or tragedy', 'The Royal Heir': 'A legitimate or hidden heir to the throne who must reclaim their birthright', 'The Mysterious Stranger': 'A enigmatic figure with unclear motives who appears at crucial moments', 'The Evil Advisor': 'A manipulative counselor who poisons the mind of those in power', 'The Knight Errant': 'A wandering warrior b

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [108]:
umap_df = doUMAP(fantasyArchetypesKeysValues, fantasyEmbeddingValues)
umap_df

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



,Archetype,Description,x,y
0,The Chosen One,A seemingly ordinary person destined for extra...,5.887012,-1.578246
1,The Wise Mentor,An elderly sage who guides the hero with wisdo...,5.850201,-0.745863
2,The Dark Lord,A powerful evil being seeking domination or de...,7.898288,0.685768
3,The Loyal Friend,A steadfast companion who supports the hero th...,5.547814,-1.052936
4,The Warrior Princess,A noble woman who defies traditional roles to ...,6.325529,-2.281577
...,...,...,...,...
83,The Cosmic Trickster,A powerful being who uses chaos and mischief t...,4.668180,1.235215
84,The Spirit Walker,A shaman who can journey through the spirit world,4.068613,1.385692
85,The Cursed Prophet,A seer whose predictions are true but never be...,6.472955,-0.331945
86,The Elemental Lord,A being who embodies and controls a force of n...,3.737264,-0.442180


In [109]:
import pandas as pd

# Add a new column 'characters' to umap_df
umap_df['characters'] = ''
umap_df['characters'] = umap_df['characters'].astype(object)

for archetype in fantasyArchetypes:
    title = archetype['title']
    description = archetype['description']
    characters = archetype['characters']  # This is already a list

    matching_rows = umap_df[(umap_df['Archetype'] == title) & (umap_df['Description'] == description)].index

    if not matching_rows.empty:
        # Assign characters directly without extra list wrapping
        umap_df.at[matching_rows[0], 'characters'] = characters
    else:
        print(f"Warning: No matching row found for title '{title}' in the umap_df.")

umap_df

,Archetype,Description,x,y,characters
0,The Chosen One,A seemingly ordinary person destined for extra...,5.887012,-1.578246,"[Harry Potter, Frodo Baggins, Paul Atreides]"
1,The Wise Mentor,An elderly sage who guides the hero with wisdo...,5.850201,-0.745863,"[Gandalf, Dumbledore, Obi-Wan Kenobi]"
2,The Dark Lord,A powerful evil being seeking domination or de...,7.898288,0.685768,"[Sauron, Voldemort, The Night King]"
3,The Loyal Friend,A steadfast companion who supports the hero th...,5.547814,-1.052936,"[Samwise Gamgee, Ron Weasley, Chewbacca]"
4,The Warrior Princess,A noble woman who defies traditional roles to ...,6.325529,-2.281577,"[Eowyn, Xena, Wonder Woman]"
...,...,...,...,...,...
83,The Cosmic Trickster,A powerful being who uses chaos and mischief t...,4.668180,1.235215,"[Q, Mr. Mxyzptlk, Coyote]"
84,The Spirit Walker,A shaman who can journey through the spirit world,4.068613,1.385692,"[Aang, John Constantine, Doctor Strange]"
85,The Cursed Prophet,A seer whose predictions are true but never be...,6.472955,-0.331945,"[Cassandra, Trelawney, The Oracle]"
86,The Elemental Lord,A being who embodies and controls a force of n...,3.737264,-0.442180,"[Poseidon, Gaia, Mother Nature]"


In [110]:
umap_df['color'] = "#FF5733"
umap_df

,Archetype,Description,x,y,characters,color
0,The Chosen One,A seemingly ordinary person destined for extra...,5.887012,-1.578246,"[Harry Potter, Frodo Baggins, Paul Atreides]",#FF5733
1,The Wise Mentor,An elderly sage who guides the hero with wisdo...,5.850201,-0.745863,"[Gandalf, Dumbledore, Obi-Wan Kenobi]",#FF5733
2,The Dark Lord,A powerful evil being seeking domination or de...,7.898288,0.685768,"[Sauron, Voldemort, The Night King]",#FF5733
3,The Loyal Friend,A steadfast companion who supports the hero th...,5.547814,-1.052936,"[Samwise Gamgee, Ron Weasley, Chewbacca]",#FF5733
4,The Warrior Princess,A noble woman who defies traditional roles to ...,6.325529,-2.281577,"[Eowyn, Xena, Wonder Woman]",#FF5733
...,...,...,...,...,...,...
83,The Cosmic Trickster,A powerful being who uses chaos and mischief t...,4.668180,1.235215,"[Q, Mr. Mxyzptlk, Coyote]",#FF5733
84,The Spirit Walker,A shaman who can journey through the spirit world,4.068613,1.385692,"[Aang, John Constantine, Doctor Strange]",#FF5733
85,The Cursed Prophet,A seer whose predictions are true but never be...,6.472955,-0.331945,"[Cassandra, Trelawney, The Oracle]",#FF5733
86,The Elemental Lord,A being who embodies and controls a force of n...,3.737264,-0.442180,"[Poseidon, Gaia, Mother Nature]",#FF5733


In [111]:
import numpy as np

def normalize_xy(df, x_col, y_col, min_xval=75, max_xval=350, min_yval=50, max_yval=350):
  """Normalizes x and y columns in a DataFrame to a specified range.

  Args:
      df: The input DataFrame.
      x_col: The name of the x column.
      y_col: The name of the y column.
      min_val: The minimum value of the normalized range.
      max_val: The maximum value of the normalized range.

  Returns:
      A new DataFrame with normalized x and y columns.
  """
  # Find the min and max of x and y in the dataframe
  x_min = df[x_col].min()
  x_max = df[x_col].max()
  y_min = df[y_col].min()
  y_max = df[y_col].max()

  # Apply min-max normalization
  df[x_col] = (df[x_col] - x_min) / (x_max - x_min) * (max_xval - min_xval) + min_xval
  df[y_col] = (df[y_col] - y_min) / (y_max - y_min) * (max_yval - min_yval) + min_yval
  return df

umap_df = normalize_xy(umap_df, 'x', 'y')
umap_df

,Archetype,Description,x,y,characters,color
0,The Chosen One,A seemingly ordinary person destined for extra...,218.566055,93.658951,"[Harry Potter, Frodo Baggins, Paul Atreides]",#FF5733
1,The Wise Mentor,An elderly sage who guides the hero with wisdo...,216.284882,145.328735,"[Gandalf, Dumbledore, Obi-Wan Kenobi]",#FF5733
2,The Dark Lord,A powerful evil being seeking domination or de...,343.202667,234.196625,"[Sauron, Voldemort, The Night King]",#FF5733
3,The Loyal Friend,A steadfast companion who supports the hero th...,197.546295,126.267365,"[Samwise Gamgee, Ron Weasley, Chewbacca]",#FF5733
4,The Warrior Princess,A noble woman who defies traditional roles to ...,245.740448,50.000000,"[Eowyn, Xena, Wonder Woman]",#FF5733
...,...,...,...,...,...,...
83,The Cosmic Trickster,A powerful being who uses chaos and mischief t...,143.036362,268.303284,"[Q, Mr. Mxyzptlk, Coyote]",#FF5733
84,The Spirit Walker,A shaman who can journey through the spirit world,105.881783,277.644073,"[Aang, John Constantine, Doctor Strange]",#FF5733
85,The Cursed Prophet,A seer whose predictions are true but never be...,254.876282,171.022552,"[Cassandra, Trelawney, The Oracle]",#FF5733
86,The Elemental Lord,A being who embodies and controls a force of n...,85.348450,164.179779,"[Poseidon, Gaia, Mother Nature]",#FF5733


In [112]:
umap_df = umap_df.rename(columns={'Archetype': 'name', 'Description': 'description', 'characters': 'tags'})
umap_df['id'] = umap_df.index
umap_df['id'] = umap_df['id'].astype(str)
umap_df

,name,description,x,y,tags,color,id
0,The Chosen One,A seemingly ordinary person destined for extra...,218.566055,93.658951,"[Harry Potter, Frodo Baggins, Paul Atreides]",#FF5733,0
1,The Wise Mentor,An elderly sage who guides the hero with wisdo...,216.284882,145.328735,"[Gandalf, Dumbledore, Obi-Wan Kenobi]",#FF5733,1
2,The Dark Lord,A powerful evil being seeking domination or de...,343.202667,234.196625,"[Sauron, Voldemort, The Night King]",#FF5733,2
3,The Loyal Friend,A steadfast companion who supports the hero th...,197.546295,126.267365,"[Samwise Gamgee, Ron Weasley, Chewbacca]",#FF5733,3
4,The Warrior Princess,A noble woman who defies traditional roles to ...,245.740448,50.000000,"[Eowyn, Xena, Wonder Woman]",#FF5733,4
...,...,...,...,...,...,...,...
83,The Cosmic Trickster,A powerful being who uses chaos and mischief t...,143.036362,268.303284,"[Q, Mr. Mxyzptlk, Coyote]",#FF5733,83
84,The Spirit Walker,A shaman who can journey through the spirit world,105.881783,277.644073,"[Aang, John Constantine, Doctor Strange]",#FF5733,84
85,The Cursed Prophet,A seer whose predictions are true but never be...,254.876282,171.022552,"[Cassandra, Trelawney, The Oracle]",#FF5733,85
86,The Elemental Lord,A being who embodies and controls a force of n...,85.348450,164.179779,"[Poseidon, Gaia, Mother Nature]",#FF5733,86


In [113]:
import json

# Convert the DataFrame to a JSON string
umap_json_string = umap_df.to_json(orient='records')

# Optionally, you can save the JSON string to a file
with open('umap_data.json', 'w') as f:
  f.write(umap_json_string)

# Optionally print or use the JSON string
umap_json_string


'[{"name":"The Chosen One","description":"A seemingly ordinary person destined for extraordinary things, often marked by prophecy or unique abilities","x":218.5660552979,"y":93.6589508057,"tags":["Harry Potter","Frodo Baggins","Paul Atreides"],"color":"#FF5733","id":"0"},{"name":"The Wise Mentor","description":"An elderly sage who guides the hero with wisdom and sometimes magic","x":216.2848815918,"y":145.3287353516,"tags":["Gandalf","Dumbledore","Obi-Wan Kenobi"],"color":"#FF5733","id":"1"},{"name":"The Dark Lord","description":"A powerful evil being seeking domination or destruction of the world","x":343.2026672363,"y":234.1966247559,"tags":["Sauron","Voldemort","The Night King"],"color":"#FF5733","id":"2"},{"name":"The Loyal Friend","description":"A steadfast companion who supports the hero through their journey","x":197.546295166,"y":126.267364502,"tags":["Samwise Gamgee","Ron Weasley","Chewbacca"],"color":"#FF5733","id":"3"},{"name":"The Warrior Princess","description":"A noble wo